In [ ]:
!pip install --upgrade typing-extensions
# !pip install --upgrade openai
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
import openai
import time
import os

def _ms_since_epoch():
    return time.perf_counter_ns() // 1000000


def set_openai_parameters(engine, max_tokens):
    # openai API setup and parameters
    openai.api_key = "sk-proj-LEu3lWTFbZ1inVRqd5E9T3BlbkFJqATqVJMZbDaGnm7nznzH"
    parameters = {
        "max_tokens": max_tokens,
        "top_p": 0,  # greedy
        "temperature": 0.5,
        "logprobs": 5,  # maximal value accorrding to https://beta.openai.com/docs/api-reference/completions/create#completions/create-logprobs, used to be 10...
        "engine": engine,
    }
    time_of_last_api_call = _ms_since_epoch()

    return parameters, time_of_last_api_call


def wait_between_predictions(time_of_last_api_call, min_ms_between_api_calls):
    if (
        cur_time := _ms_since_epoch()
    ) <= time_of_last_api_call + min_ms_between_api_calls:
        ms_to_sleep = min_ms_between_api_calls - (cur_time - time_of_last_api_call)
        time.sleep(ms_to_sleep / 1000)
    time_of_last_api_call = _ms_since_epoch()


def predict_sample_openai_gpt(
    example,
    prompt,
    min_ms_between_api_calls: int = 500,
    engine: str = "text-davinci-003",
    max_tokens: int = 100,
):
    parameters, time_of_last_api_call = set_openai_parameters(engine, max_tokens)
    parameters["prompt"] = prompt

    # OpenAI limits us to 3000 calls per minute:
    # https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits
    # that is why the default value of min_ms_between_api_calls is 20
    wait_between_predictions(time_of_last_api_call, min_ms_between_api_calls)

    response = openai.Completion.create(**parameters)

    if response is None:
        raise Exception("Response from OpenAI API is None.")

    # build output data
    prediction = dict()
    prediction["input"] = prompt
    prediction["prediction"] = response.choices[0].text.strip().strip(".")  # type:ignore

    # build output metadata
    metadata = example.copy()  # dict()
    metadata["logprobs"] = response.choices[0]["logprobs"]  # type:ignore
    # "finish_reason" is located in a slightly different location in opt
    if "opt" in engine:
        finish_reason = response.choices[0]["logprobs"][  # type:ignore
            "finish_reason"
        ]
    else:
        finish_reason = response.choices[0]["finish_reason"]  # type:ignore
    metadata["finish_reason"] = finish_reason
    if "opt" not in engine:
        # From the OpenAI API documentation it's not clear what "index" is, but let's keep it as well
        metadata["index"] = response.choices[0]["index"]  # type:ignore

    prediction["metadata"] = metadata

    return prediction

def predict_sample_openai_chatgpt(
    prompt,
    img_url,
    min_ms_between_api_calls: int = 50000,
    engine: str = "gpt-4o",
    max_tokens: int = 100,
):
    parameters, time_of_last_api_call = set_openai_parameters(engine, max_tokens)
    parameters["prompt"] = prompt

    # OpenAI limits us to 3000 calls per minute:
    # https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits
    wait_time = 5
    time.sleep(wait_time)
    try:
        response = openai.ChatCompletion.create(model=engine, messages=[{"role": "user", "content": [{"type": "text", "text":prompt},{
          "type": "image_url",
          "image_url": {"url": f"data:image/jpeg;base64,{img_url}"
}}]}], temperature=parameters['temperature'], top_p=parameters['top_p'])
    except openai.error.RateLimitError as e:
        wait_time = 10
        print(f"Rate limit reached. Waiting {wait_time} seconds.")
        time.sleep(wait_time)

        response = openai.ChatCompletion.create(model=engine, messages=[{"role": "user", "content": prompt}],
                                                temperature=parameters['temperature'], top_p=parameters['top_p'])

    if response is None:
        raise Exception("Response from OpenAI API is None.")

    # build output data
    prediction = dict()
    prediction["input"] = prompt
    prediction["prediction"] = response.choices[0].message['content']  # type:ignore

    return prediction

def gpt4_estimetion(url,question):

  prompt = f"Answer the following question based on the image: {question}"
#   prompt= f"""
# generate a caption for the following image
#   """
  gpt4_prediction = predict_sample_openai_chatgpt(prompt,url)
  return gpt4_prediction['prediction']

In [ ]:
!pip install -q git-lfs
!git clone https://huggingface.co/spaces/nlphuji/whoops-explorer-analysis
!pip install -q datasets
from datasets import load_dataset
# import gradio as gr
import os
import random

wmtis = load_dataset("nlphuji/wmtis-identify")['test']

Cloning into 'whoops-explorer-analysis'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (7/7), done.
remote: Total 26 (delta 6), reused 6 (delta 6), pack-reused 19 (from 1)
Unpacking objects: 100% (26/26), 5.40 KiB | 691.00 KiB/s, done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarr

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for nlphuji/wmtis-identify contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nlphuji/wmtis-identify.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating test split: 0 examples [00:00, ? examples/s]

dropping natural url


In [ ]:
print(len(wmtis))
print(wmtis[45])

102
{'natural_image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=860x614 at 0x78B5BBC2E860>, 'normal_image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024 at 0x78B5BBC2EEF0>, 'strange_image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x78B5845894E0>, 'natural_hash': '2b0852954e5fd658b09586e02cdf6d5b2ed68d68989b427c4688dba3d3e457d7', 'normal_hash': 'da0c39cc7246f2228238fad63ec8a405a9b6a35eaf3441bd89572ba6777950b9', 'strange_hash': 'ffad7c0deee280ce9578dd0f3094e517b0a1fa7d4ba47c09201c09d5c5eb8494', 'natural_image_caption': 'a traffic light on a black stand against a white background', 'normal_image_caption': 'a traffic light on a pole', 'strange_image_caption': 'a green traffic light on a gray background', 'rating_natural': 1, 'rating_normal': 1, 'rating_strange': 0, 'comments_natural': 'nan', 'comments_normal': 'nan', 'comments_strange': 'Missing "all green light"'}


In [ ]:
import ast
import pandas as pd
import re

df = pd.read_csv("fixedLabels_full.csv")
questions = []
for index, row in df.iterrows():
  full_list = row['full']
  match = re.search(r"\[(.*?),", full_list)
  if match:
      question = match.group(1)
      questions.append(question)
  else:
    print(question)

print(len(questions))
print(questions[len(questions) - 1])




102
"ignore the image and answer: not valid"


In [ ]:
import base64
from io import BytesIO
from openai.error import InvalidRequestError

gpt4_captions ={'natural_caption':[],'normal_caption':[],'strange_caption':[]}
question_index = -1
for record in wmtis:
  question_index += 1
  natural_image = record['natural_image']
  normal_image = record['normal_image']
  strange_image = record['strange_image']
  # print(record)

  # buffered = BytesIO()
  # natural_image.save(buffered, format="PNG")
  # natural_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
  # try:
  #   natural_caption = gpt4_estimetion(natural_str)
  #   print(natural_caption)
  #   gpt4_captions['natural_caption'].append(natural_caption)
  # except InvalidRequestError as e:
  #   print(f"Failed to get caption: {e}")


  buffered = BytesIO()
  normal_image.save(buffered, format="PNG")
  normal_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
  try:
    print(questions[question_index])
    normal_answer = gpt4_estimetion(normal_str, questions[question_index])
    print(normal_answer)
    gpt4_captions['normal_caption'].append(normal_answer)
  except InvalidRequestError as e:
    print(f"Failed to get caption: {e}")

  # buffered = BytesIO()
  # strange_image.save(buffered, format="PNG")
  # strange_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
  # try:
  #   print(questions[question_index])
  #   strange_answer = gpt4_estimetion(strange_str,questions[question_index])
  #   print(strange_answer)
  #   gpt4_captions['strange_caption'].append(strange_answer)
  # except InvalidRequestError as e:
  #   print(f"Failed to get caption: {e}")


'what is the snow plow plowing in the desert?'
The snow plow in the image is plowing snow, not sand in a desert. The image shows a snow-covered environment, indicating that the snow plow is clearing snow from a road or path.
'Where is a life saving buoy located on the ocean floor?'
The image shows a life-saving buoy floating on the surface of the water, not located on the ocean floor.
'A boat with crocheted sails travels over what?'
The boat with crocheted sails travels over the sea.
'the toddler wrinting advanced math equations on what?'
The image does not depict a toddler writing advanced math equations. Instead, it shows an adult man in a suit and glasses writing advanced math equations on a chalkboard.
'Who is riding on a horse with his elves?'
The image depicts Santa Claus riding in a sleigh pulled by reindeer. There are no elves visible in the image.
'What is the pregnant woman smoking?'
The pregnant woman in the image is not smoking. She is holding a plastic water bottle.
'what 

In [ ]:
print(len(gpt4_captions['normal_caption']))

101


In [ ]:

# natural_df = pd.DataFrame(gpt4_captions['natural_caption'])
# natural_df.to_csv('natural_caption.csv', index=False)  # index=False to avoid writing row numbers

normal_df = pd.DataFrame(gpt4_captions['normal_caption'])
normal_df.to_csv('normal_answers.csv', index=False)  # index=False to avoid writing row numbers

# strange_df = pd.DataFrame(gpt4_captions['strange_caption'])
# strange_df.to_csv('strange_answers.csv', index=False)  # index=False to avoid writing row numbers